# DNN Model Notebook

```md
@authors: miguelrocha and Grupo 03
```

In [1]:
# Notebook Imports

from models.dnn_model import NeuralNetwork, hyperparameter_optimization
from helpers.dataset import Dataset
from helpers.enums import ModelRunMode
from helpers.layers import DenseLayer
from helpers.activation import ReLUActivation, SigmoidActivation
from helpers.losses import BinaryCrossEntropy
from helpers.optimizer import Optimizer
from helpers.metrics import accuracy
from helpers.enums import ModelType
from helpers.model import save_dnn_model

In [2]:
# Model run mode
# Options: 
#   ModelRunMode.TRAIN.value            (Train the model)
#   ModelRunMode.CLASSIFY.value         (Classify data)
mode = ModelRunMode.CLASSIFY.value
# Prefix for saving the model files
model_prefix = "dnn_model_1"

In [3]:
# Parameters cell
if mode == ModelRunMode.TRAIN.value:
    # TRAIN mode: Set parameters for training
    input_csv = "../tarefa_1/clean_input_datasets/dataset1_enh_inputs.csv"              # CSV file with training inputs (ID, Text)
    output_csv = "../tarefa_1/clean_output_datasets/dataset1_enh_outputs.csv"           # CSV file with training outputs (ID, Label)
    test_size = 0.3                                                                     # Proportion of the dataset to use as test data
    verbose = True                                                                      # Verbosity level enabler
    random_state=42                                                                     # Seed for reproducible dataset splitting
    max_vocab_size=None                                                                 # Maximum vocabulary size (None implies no limit)
    min_freq=1                                                                          # Minimum frequency required for a word to be included in the vocabulary
elif mode == ModelRunMode.CLASSIFY.value:
    # CLASSIFY mode: Set parameters for classification
    input_csv = "../tarefa_2/classify_input_datasets/dataset2_inputs.csv"               # CSV file with texts for prediction (ID, Text)
    output_csv = "../tarefa_2/classify_output_datasets/dataset2_outputs_lr_model.csv"   # CSV file to store prediction result
else:
    print("The selected option is not valid. Options: \"train\" or \"classify\"!")
    SystemExit()

In [4]:
if mode == ModelRunMode.TRAIN.value:
    # Load datasets using TF-IDF vectorization
    X_train, y_train, X_test, y_test, vocab, idf = Dataset.prepare_train_test_tfidf(input_csv=input_csv, output_csv=output_csv, test_size=test_size, random_state=random_state, max_vocab_size=max_vocab_size, min_freq=min_freq)

    # Create Dataset objects for training and test data
    train_ds = Dataset(X=X_train, Y=y_train)
    test_ds = Dataset(X=X_test, Y=y_test)

    # Display dimensions of the training and test datasets
    print(f"Train set has {train_ds.nrows()} rows and {train_ds.ncols()} columns")
    print(f"Test set has {test_ds.nrows()} rows and {test_ds.ncols()} columns\n")

    # Define hyperparameter grids for tuning
    epochs_list = [20, 40, 60, 80, 100, 200, 500]
    batch_size_list = [16, 32, 64]
    learning_rate_list = [0.001, 0.01, 0.1]
    momentum_list = [0.1, 0.5, 0.9]
    hidden_layers_list = [[32], [64, 32], [64, 64], [128, 64]]
    dropout_list = [0.0, 0.01, 0.1, 0.5]

    # Perform hyperparameter tuning using the training and validation sets
    print("Starting hyperparameter tuning...")
    best_params = hyperparameter_optimization(train_ds, test_ds, epochs_list, batch_size_list, learning_rate_list, momentum_list, hidden_layers_list, dropout_list, n_iter=50)
    print("\nBest hyperparameters:", best_params)

    # Retrain DNN model on the full training data using the best hyperparameters
    final_model = NeuralNetwork(epochs=best_params['epochs'], batch_size=best_params['batch_size'], optimizer=Optimizer(learning_rate=best_params['learning_rate'], momentum=best_params['momentum']), verbose=verbose, loss=BinaryCrossEntropy,  metric=accuracy)

    # Build model Layers
    n_features = train_ds.X.shape[1]
    for i, units in enumerate(best_params['n_hidden']):
        if i == 0:
            final_model.add(DenseLayer(units, (n_features,),dropout_rate=best_params['dropout_rate']))
        else:
            final_model.add(DenseLayer(units,dropout_rate=best_params['dropout_rate']))
        final_model.add(ReLUActivation())
        
    final_model.add(DenseLayer(1))
    final_model.add(SigmoidActivation())

    # Fit model
    final_model.fit(train_ds)
    final_model.predict(test_ds)
    out = final_model.predict(test_ds)

    # Evaluate the final model on the test set
    acc = final_model.score(test_ds, out)
    print("Test Accuracy:", acc)

    # Save the model, plus vocab & idf, so classification can replicate
    save_dnn_model(dnn=final_model, vocab=vocab, idf=idf, model_prefix=model_prefix)
    print(f"Model saved with prefix {model_prefix}")

if mode == ModelRunMode.CLASSIFY.value:
    # Classify new texts using the saved model
    Dataset.classify_texts(input_csv=input_csv, output_csv=output_csv, neural_net_class=NeuralNetwork, model_type=ModelType.DNN ,model_prefix=model_prefix)

DNN Model loaded from 'dnn_model_weights\dnn_model_1_architecture.json'
[DNN] Predictions saved to ../tarefa_2/classify_output_datasets/dataset2_outputs_lr_model.csv
